In [1]:
import sys
import os
import time

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

/home/yuncong/MouseBrainAtlas/src/utilities/utilities2015.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7

Setting environment for Precision WorkStation
{'DEMO998': 0.46}


ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
No vtk
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Seems you are using operation INIs to provide cropbox.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Seems you are using operation INIs to provide cropbox.


In [107]:
def get_structure_contours_from_structure_volumes_v3(volumes, stack, sections, 
                                                     resolution, level, sample_every=1,
                                                    use_unsided_name_as_key=False):
    """
    Re-section atlas volumes and obtain structure contours on each section.
    Resolution of output contours are in volume resolution.
    v3 supports multiple levels.

    Args:
        volumes (dict of (3D array, 3-tuple)): {structure: (volume, origin_wrt_wholebrain)}. volume is a 3d array of probability values.
        sections (list of int):
        resolution (int): resolution of input volumes.
        level (float or dict or dict of list): the cut-off probability at which surfaces are generated from probabilistic volumes. Default is 0.5.
        sample_every (int): how sparse to sample contour vertices.

    Returns:
        Dict {section: {name_s: contour vertices wrt_alignedBrainstemCrop_rawResol}}.
    """

    from collections import defaultdict
    
    structure_contours_wrt_alignedBrainstemCrop_rawResol = defaultdict(lambda: defaultdict(dict))

    converter = CoordinatesConverter(stack=stack, section_list=metadata_cache['sections_to_filenames'][stack].keys())

    converter.register_new_resolution('structure_volume', resol_um=convert_resolution_string_to_um(resolution=resolution, stack=stack))
    converter.register_new_resolution('image', resol_um=convert_resolution_string_to_um(resolution='raw', stack=stack))
    
    for name_s, (structure_volume_volResol, origin_wrt_wholebrain_volResol) in volumes.iteritems():

        converter.derive_three_view_frames(name_s, 
        origin_wrt_wholebrain_um=convert_resolution_string_to_um(resolution=resolution, stack=stack) * origin_wrt_wholebrain_volResol,
        zdim_um=convert_resolution_string_to_um(resolution=resolution, stack=stack) * structure_volume_volResol.shape[2])

        positions_of_all_sections_wrt_structureVolume = converter.convert_frame_and_resolution(
        p=np.array(sections)[:,None],
        in_wrt=('wholebrain', 'sagittal'), in_resolution='section',
        out_wrt=(name_s, 'sagittal'), out_resolution='structure_volume')[..., 2].flatten()
            
        structure_ddim = structure_volume_volResol.shape[2]
        
        valid_mask = (positions_of_all_sections_wrt_structureVolume >= 0) & (positions_of_all_sections_wrt_structureVolume < structure_ddim)
        if np.count_nonzero(valid_mask) == 0:
#             sys.stderr.write("%s, valid_mask is empty.\n" % name_s)
            continue

        positions_of_all_sections_wrt_structureVolume = positions_of_all_sections_wrt_structureVolume[valid_mask]
        positions_of_all_sections_wrt_structureVolume = np.round(positions_of_all_sections_wrt_structureVolume).astype(np.int)
        
        if isinstance(level, dict):
            level_this_structure = level[name_s]
        else:
            level_this_structure = level

        if isinstance(level_this_structure, float):
            level_this_structure = [level_this_structure]
                        
        for one_level in level_this_structure:

            contour_2d_wrt_structureVolume_sectionPositions_volResol = \
            find_contour_points_3d(structure_volume_volResol >= one_level,
                                    along_direction='sagittal',
                                    sample_every=sample_every,
                                    positions=positions_of_all_sections_wrt_structureVolume)

            for d_wrt_structureVolume, cnt_uv_wrt_structureVolume in contour_2d_wrt_structureVolume_sectionPositions_volResol.iteritems():

                contour_3d_wrt_structureVolume_volResol = np.column_stack([cnt_uv_wrt_structureVolume, np.ones((len(cnt_uv_wrt_structureVolume),)) * d_wrt_structureVolume])

    #             contour_3d_wrt_wholebrain_uv_rawResol_section = converter.convert_frame_and_resolution(
    #                 p=contour_3d_wrt_structureVolume_volResol,
    #                 in_wrt=(name_s, 'sagittal'), in_resolution='structure_volume',
    #                 out_wrt=('wholebrain', 'sagittal'), out_resolution='image_image_section')

                contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section = converter.convert_frame_and_resolution(
                    p=contour_3d_wrt_structureVolume_volResol,
                    in_wrt=(name_s, 'sagittal'), in_resolution='structure_volume',
                    out_wrt=('wholebrainXYcropped', 'sagittal'), out_resolution='image_image_section')

                assert len(np.unique(contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[:,2])) == 1
                sec = int(contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[0,2])

                if use_unsided_name_as_key:
                    name = convert_to_unsided_label(name_s)
                else:
                    name = name_s

                structure_contours_wrt_alignedBrainstemCrop_rawResol[sec][name][one_level] = contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[..., :2]
        
    return structure_contours_wrt_alignedBrainstemCrop_rawResol


In [108]:
per_structure_alignment_spec = load_json('../../demo/demo_visualization_per_structure_alignment_spec.json')

In [109]:
simpleGlobal_alignment_spec = load_json('../../demo/demo_visualization_global_alignment_spec.json')

In [110]:
structure_list = ['12N']

In [111]:
section_margin_um = 1000.
section_margin = int(section_margin_um / SECTION_THICKNESS)

stack = 'DEMO998'
# stack = brain_f_spec['name']
# valid_secmin = np.min(metadata_cache['valid_sections'][stack])
# valid_secmax = np.max(metadata_cache['valid_sections'][stack])
valid_secmin = 1
valid_secmax = 999

In [112]:
auto_contours_all_sec_all_structures_all_levels = defaultdict(lambda: defaultdict(dict))
simple_global_contours_all_sec_all_structures_all_levels = defaultdict(lambda: defaultdict(dict))

In [116]:
for structure_m in structure_list:

    ####################################################
    
    local_alignment_spec = per_structure_alignment_spec[structure_m]
    
    vo = DataManager.load_transformed_volume_v2(alignment_spec=local_alignment_spec, 
                                                return_origin_instead_of_bbox=True,
                                               structure=structure_m)

    # prep2 because at end of get_structure_contours_from_structure_volumes_v2 we used wholebrainXYcropped
    registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners_atlasResol = \
    load_json(os.path.join(DATA_ROOTDIR, 'CSHL_simple_global_registration', stack + '_registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners.json'))

    (_, _, secmin), (_, _, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners_atlasResol[structure_m]
    print secmin, secmax

    atlas_structures_wrt_wholebrainXYcropped_sections = \
    range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax) + 1)
    
#     atlas_structures_wrt_wholebrainWithMargin_sections = \
#     range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax) + 1)

    levels = [0.1, 0.25, 0.5, 0.75, 0.99]

#     auto_contours_all_sections_one_structure_all_levels = \
#     get_structure_contours_from_structure_volumes_v3(volumes={structure_m: vo}, stack=stack, 
#                                                      sections=atlas_structures_wrt_wholebrainWithMargin_sections,
#                                                     resolution='10.0um', level=levels, sample_every=5)

#     print atlas_structures_wrt_wholebrainWithMargin_sections

    auto_contours_all_sections_one_structure_all_levels = \
    get_structure_contours_from_structure_volumes_v3(volumes={structure_m: vo}, stack=stack, 
                                                     sections=atlas_structures_wrt_wholebrainXYcropped_sections,
                                                    resolution='10.0um', level=levels, sample_every=5)

    print atlas_structures_wrt_wholebrainXYcropped_sections
    print vo[0].shape, vo[1]

    for sec, contours_one_structure_all_levels in sorted(auto_contours_all_sections_one_structure_all_levels.items()):
        if not is_invalid(sec=sec, stack=stack):
            for name_s, cnt_all_levels in contours_one_structure_all_levels.items():
                for level, cnt in cnt_all_levels.iteritems():
                    auto_contours_all_sec_all_structures_all_levels[sec][name_s][level] = cnt.astype(np.int)

    ####################################################


    simpleGlobal_vo_wrt_wholebrain = DataManager.load_transformed_volume_v2(alignment_spec=simpleGlobal_alignment_spec, 
                                                             return_origin_instead_of_bbox=True,
                                                            structure=structure_m)

    simpleGlobal_contours_all_sections_one_structure_all_levels = \
    get_structure_contours_from_structure_volumes_v3(volumes={structure_m: simpleGlobal_vo_wrt_wholebrain}, stack=stack, 
                                                     sections=atlas_structures_wrt_wholebrainXYcropped_sections,
                                                    resolution='10.0um', level=levels, sample_every=5)
    
    for sec, contours_one_structure_all_levels in sorted(simpleGlobal_contours_all_sections_one_structure_all_levels.items()):
        if not is_invalid(sec=sec, stack=stack):
            for name_s, cnt_all_levels in contours_one_structure_all_levels.items():
                for level, cnt in cnt_all_levels.iteritems():
                    simple_global_contours_all_sec_all_structures_all_levels[sec][name_s][level] = cnt.astype(np.int)


209 251


No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Seems you are using operation INIs to provide cropbox.
2 contours of reconstructed volume is found at position 111 ([45, 1]). Use the longest one.
3 contours of reconstructed volume is found at position 125 ([16, 1, 1]). Use the longest one.
2 contours of reconstructed volume is found at position 49 ([36, 3]). Use the longest one.
2 contours of reconstructed volume is found at position 55 ([32, 7]). Use the longest one.


[159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302]
(100, 137, 146) [1100.  397.  414.]


No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Seems you are using operation INIs to provide cropbox.


In [117]:
for sec in [225]:


    for version in ['NtbNormalizedAdaptiveInvertedGammaJpeg']:
#         for version in ['grayJpeg']:
#         for version in ['NtbNormalizedAdaptiveInvertedGammaJpeg', 'CHATJpeg']:
        
        try:
            img = DataManager.load_image_v2(stack=stack, prep_id=2, resol='raw', version=version, section=sec)

            viz = gray2rgb(img)

            # Draw the locally aligned structure contours in COLOR
            for name_s, cnt_all_levels in auto_contours_all_sec_all_structures_all_levels[sec].iteritems():

                for level, cnt in cnt_all_levels.iteritems():
                    cv2.polylines(viz, [cnt.astype(np.int)], isClosed=True, 
                                  color=LEVEL_TO_COLOR_LINE[level], thickness=10)
            
            # Draw the simple globally aligned structure contours in WHITE
            for name_s, cnt_all_levels in simple_global_contours_all_sec_all_structures_all_levels[sec].iteritems():

                for level, cnt in cnt_all_levels.iteritems():
                    cv2.polylines(viz, [cnt.astype(np.int)], isClosed=True, 
                                  color=(255,255,255), 
                                  thickness=10)

    # #             # Add CHAT contour
    #             if sec in chat_contours_all_sections_all_structures_all_levels:
    #                 chat_cnt = chat_contours_all_sections_all_structures_all_levels[sec][name_s][.5]
    #                 cv2.polylines(viz, [chat_cnt.astype(np.int)], isClosed=True, color=(255,255,255), thickness=20)

    #             fp = os.path.join('/home/yuncong/' + stack + '_atlas_aligned_multilevel_all_structures', version, stack + '_' + version + '_' + ('%03d' % sec) + '.jpg')
    #             print fp
    #             create_parent_dir_if_not_exists(fp)
    #             imsave(fp, viz)

#             fp = os.path.join(ROOT_DIR, 'CSHL_registration_visualization', 
#                               stack + '_atlas_aligned_multilevel_down16_all_structures', 
#                               version, stack + '_' + version + '_' + ('%03d' % sec) + '.jpg')    
#             create_parent_dir_if_not_exists(fp)
#             imsave(fp, viz[::16, ::16])
#  #           upload_to_s3(fp)
            
        except:
            pass


Trying to load /home/yuncong/MouseBrainAtlas/demo/demo_data/CSHL_data_processed/DEMO998/DEMO998_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg/MD662&661-F81-2017.06.06-12.44.40_MD661_2_0242_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg.jpg


In [118]:
display_image(viz[::16, ::16])

/home/yuncong/MouseBrainAtlas/src/registration/tmp.jpg